In [1]:
#!pip3 install ray torch torchvision tabulate tensorboard
#!pip3 install 'ray[rllib]'
#!pip3 install ray

In [2]:
import warnings
warnings.filterwarnings('ignore')

import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

from models import VisualEncoder
from train import *

In [3]:
class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 512
        self.encoder = VisualEncoder(features_dim)
        #self.encoder.load_state_dict(
        #    torch.load("Visual Autoencoder weights and models/encoder_weigths.pth", map_location=torch.device('cpu'))
        #)
        self.action_head = nn.Linear(features_dim, action_space.n)
        self.value_head = nn.Linear(features_dim, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
            
        features = self.encoder(obs)
        action = self.action_head(features)
        self.last_value = self.value_head(features).squeeze(1)
        return action, state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value

In [4]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=1000)
    env.update_taskset(TaskSet(preset=['C22']))
    env = PovOnlyWrapper(env)
    env = IgluActionWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

tune.run(PPOTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 1,
             "sgd_minibatch_size": 256,
             "clip_param": 0.2,
             "entropy_coeff": 0.01,
             "lambda": 0.95,
             "train_batch_size": 1000,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "PPO C22 not pretrained"
                  }
              }

        },
        loggers=[WandbLogger])#callbacks=[
        #    CustomLoggerCallback(),
        #])

Trial name,status,loc
PPO_my_env_8f2a6_00000,PENDING,


2021-09-16 11:16:37,912	INFO wandb.py:170 -- Already logged into W&B.
wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
wandb: WARNING Tried to auto resume run with id 7c454_00000 but id 8f2a6_00000 is set.
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
(pid=16561) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=16561)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=16561) 2021-09-16 11:16:43,997	WARNING ppo.py:143 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=3 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 1333.
(pid=16561) 2021-09-16 11:16:43,997	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=16561) 2021-09-16 11:16:43,997	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=16555) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=16555)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=16560) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casti

Result for PPO_my_env_8f2a6_00000:
  agent_timesteps_total: 7998
  custom_metrics: {}
  date: 2021-09-16_11-19-58
  done: false
  episode_len_mean: 1000.0
  episode_media: {}
  episode_reward_max: 1.0
  episode_reward_mean: 0.16666666666666666
  episode_reward_min: 0.0
  episodes_this_iter: 6
  episodes_total: 6
  experiment_id: 43f4be814be443d08f2f6e093d3d4a7f
  hostname: cdsserver
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.20000000000000004
          cur_lr: 5.0000000000000016e-05
          entropy: 2.8881016077533843
          entropy_coeff: 0.01
          kl: 0.002688916184766008
          policy_loss: -0.005954116642955811
          total_loss: -0.029144969095490993
          vf_explained_var: -0.8339848518371582
          vf_loss: 0.005152379287163043
    num_agent_steps_sampled: 7998
    num_agent_steps_trained: 7998
    num_steps_sampled: 7998
    num_steps_trained: 7998
  iterations_since_restore: 1
  node_ip: 192.168.1.96
  num

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_8f2a6_00000,RUNNING,192.168.1.96:16561,1,184.435,7998,0.166667,1,0,1000


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_8f2a6_00000,RUNNING,192.168.1.96:16561,85,6114.02,679830,1.38,4,-2,996.19


Result for PPO_my_env_8f2a6_00000:
  agent_timesteps_total: 687828
  custom_metrics: {}
  date: 2021-09-16_13-00-10
  done: false
  episode_len_mean: 996.19
  episode_media: {}
  episode_reward_max: 4.0
  episode_reward_mean: 1.44
  episode_reward_min: -2.0
  episodes_this_iter: 9
  episodes_total: 690
  experiment_id: 43f4be814be443d08f2f6e093d3d4a7f
  hostname: cdsserver
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.22500000000000006
          cur_lr: 5.0000000000000016e-05
          entropy: 2.3273062923903107
          entropy_coeff: 0.01
          kl: 0.016470402905133446
          policy_loss: -0.04300591170597541
          total_loss: -0.03127206518835518
          vf_explained_var: -0.08418731391429901
          vf_loss: 0.0313010694485681
    num_agent_steps_sampled: 687828
    num_agent_steps_trained: 687828
    num_steps_sampled: 687828
    num_steps_trained: 687828
  iterations_since_restore: 86
  node_ip: 192.168.1.96
  num_hea

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_8f2a6_00000,RUNNING,192.168.1.96:16561,86,6193.36,687828,1.44,4,-2,996.19


Result for PPO_my_env_8f2a6_00000:
  agent_timesteps_total: 695826
  custom_metrics: {}
  date: 2021-09-16_13-01-26
  done: false
  episode_len_mean: 996.19
  episode_media: {}
  episode_reward_max: 4.0
  episode_reward_mean: 1.42
  episode_reward_min: -2.0
  episodes_this_iter: 6
  episodes_total: 696
  experiment_id: 43f4be814be443d08f2f6e093d3d4a7f
  hostname: cdsserver
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.22500000000000006
          cur_lr: 5.0000000000000016e-05
          entropy: 2.37692408023342
          entropy_coeff: 0.01
          kl: 0.015367304387735058
          policy_loss: -0.04452828146487234
          total_loss: -0.04383521518239411
          vf_explained_var: -0.2191319763660431
          vf_loss: 0.021004663283641884
    num_agent_steps_sampled: 695826
    num_agent_steps_trained: 695826
    num_steps_sampled: 695826
    num_steps_trained: 695826
  iterations_since_restore: 87
  node_ip: 192.168.1.96
  num_heal

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_8f2a6_00000,RUNNING,192.168.1.96:16561,87,6269.01,695826,1.42,4,-2,996.19
